In [1]:
# TODO: will change to use "pip install -e ." for developing the package
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import time
import re

import pandas as pd
from collections import OrderedDict

import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from deck_crawler.parse_deck import parse_events_from_official
from deck_crawler.parse_deck import reassign_category
from deck_crawler.utils.translator import translate_deck

In [4]:
# flags and parameters
RUN_PARSE = False
DEBUG_PARSE = False

NUM_RESULT_PAGE = 3
NUM_EVENT_PAGE = 1000
NUM_DECK_PAGE = 2

START_DATE = "2022年01月01日"  # include
END_DATE = "2023年01月16日"  # include

TARGET_CATEGORY = "ルナトーン"

In [5]:
# create some folders
EXCEL_FOLDER = "excel"
DB_FOLDER = "deck_db"
folders = [EXCEL_FOLDER, DB_FOLDER]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

if not os.path.exists(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}"):
    os.makedirs(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}")

In [6]:
# loading db
decks = {}
store_file_name = "deck_db/test.json"
if not DEBUG_PARSE:
    store_file_name = "deck_db/store.json"
    if os.path.exists(store_file_name):
        with open(store_file_name, "r") as f:
            decks = json.load(f)

decks = reassign_category(decks)
print("categories in previous result:")
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])
print("\n")
print("number of decks in the previous result:")
print(len(store_code_list))

categories in previous result:
dict_keys(['ルナトーン', 'アル_ジュラルドン', 'かがやくムゲンダイナ', 'そらをとぶピカチュウVMAX', 'アル_レジドラゴVSTAR', 'Other_ギラティナVSTAR', 'ルギアVSTAR', 'レジエレキVMAX', 'Other_Lost', 'LTB', 'こくばバドレックスVMAX', 'アル_そらをとぶピカチュウ', 'キュレムVMAX', 'クロススイッチャー', 'カラマネロVMAX', 'オリジンパルキアVSTAR', 'ガラル マタドガス', 'LTB_空の封印石_other', 'LTB_ヤミラミ_リザードン', 'プテラVSTAR', 'アローラロコンVSTAR', 'ヒスイ ゾロアークVSTAR', 'アルセウス裏工作', 'ロトムVSTAR', 'レジドラゴVSTAR', 'ハピナスV', 'ミュウツーV-UNION', 'LOST_ギラティナVSTAR', 'オリジンディアルガVSTAR', 'LTB_リザードン', 'LTB_ウッウ', 'LTB_カイオーガ', 'LTB_カイリュー', 'LTB_空の封印石_水超鋼雷', 'LTB_空の封印石_草超鋼雷', 'LTB_空の封印石_水超闘雷', 'LTB_空の封印石_水超雷', 'LTB_空の封印石_水草超雷', 'LTB_空の封印石_水草超闘雷', 'LTB_空の封印石_水超鋼闘雷', 'LTB_空の封印石_水超鋼', 'LTB_空の封印石_超鋼雷', 'LTB_空の封印石_水草超', 'LTB_空の封印石_草超闘雷', 'LTB_空の封印石_草超鋼闘雷', 'LTB_空の封印石_水草超鋼雷', 'LTB_空の封印石_水草超鋼闘雷', 'LTB_空の封印石_超鋼闘雷', 'LTB_空の封印石_超闘雷', 'ダークライVSTAR', 'ミュウVMAX', 'ゾロア', 'ムゲンダイナVMAX', 'ヒスイ ダイケンキVSTAR', 'レジ', 'others', 'ゾロア_ウインディ'])


number of decks in the previous result:
7112


In [7]:
# parse events
t1 = time.time()
if RUN_PARSE or DEBUG_PARSE:
    parse_events_from_official(
        decks,
        store_code_list,
        result_page_limit=NUM_RESULT_PAGE,
        event_page_limit=NUM_EVENT_PAGE,
        deck_page_limit=NUM_DECK_PAGE,
    )
t2 = time.time()

print()
print(f"{t2 - t1} seconds")


0.00013518333435058594 seconds


In [8]:
# save to json
with open(store_file_name, "w") as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [9]:
# show all categories we have
total = 0
for k in decks.keys():
    print(f"[{k}]:\n{len(decks[k])}\n")
    total += len(decks[k])

print(total)

[ルナトーン]:
33

[アル_ジュラルドン]:
346

[かがやくムゲンダイナ]:
123

[そらをとぶピカチュウVMAX]:
400

[アル_レジドラゴVSTAR]:
18

[Other_ギラティナVSTAR]:
45

[ルギアVSTAR]:
1456

[レジエレキVMAX]:
104

[Other_Lost]:
146

[LTB]:
789

[こくばバドレックスVMAX]:
14

[アル_そらをとぶピカチュウ]:
258

[キュレムVMAX]:
145

[クロススイッチャー]:
257

[カラマネロVMAX]:
6

[オリジンパルキアVSTAR]:
246

[ガラル マタドガス]:
301

[LTB_空の封印石_other]:
6

[LTB_ヤミラミ_リザードン]:
141

[プテラVSTAR]:
37

[アローラロコンVSTAR]:
52

[ヒスイ ゾロアークVSTAR]:
45

[アルセウス裏工作]:
58

[ロトムVSTAR]:
21

[レジドラゴVSTAR]:
8

[ハピナスV]:
91

[ミュウツーV-UNION]:
20

[LOST_ギラティナVSTAR]:
187

[オリジンディアルガVSTAR]:
25

[LTB_リザードン]:
3

[LTB_ウッウ]:
2

[LTB_カイオーガ]:
163

[LTB_カイリュー]:
47

[LTB_空の封印石_水超鋼雷]:
82

[LTB_空の封印石_草超鋼雷]:
5

[LTB_空の封印石_水超闘雷]:
55

[LTB_空の封印石_水超雷]:
14

[LTB_空の封印石_水草超雷]:
2

[LTB_空の封印石_水草超闘雷]:
2

[LTB_空の封印石_水超鋼闘雷]:
30

[LTB_空の封印石_水超鋼]:
3

[LTB_空の封印石_超鋼雷]:
8

[LTB_空の封印石_水草超]:
1

[LTB_空の封印石_草超闘雷]:
1

[LTB_空の封印石_草超鋼闘雷]:
4

[LTB_空の封印石_水草超鋼雷]:
2

[LTB_空の封印石_水草超鋼闘雷]:
1

[LTB_空の封印石_超鋼闘雷]:
2

[LTB_空の封印石_超闘雷]:
1

[ダークライVSTAR]:
13

[ミュウVMAX]:
599

[ゾロア]:
65


In [10]:
# show categories we have in the period
number_decks = 0
for k in decks.keys():
    deck_cnt = 0
    for deck in decks[k]:
        if deck["date"] >= START_DATE and deck["date"] <= END_DATE:
            deck_cnt += 1

    print(f"{k}\t{deck_cnt}\n")
    if k == TARGET_CATEGORY:
        number_decks = deck_cnt

ルナトーン	33

アル_ジュラルドン	346

かがやくムゲンダイナ	123

そらをとぶピカチュウVMAX	400

アル_レジドラゴVSTAR	18

Other_ギラティナVSTAR	45

ルギアVSTAR	1456

レジエレキVMAX	104

Other_Lost	146

LTB	789

こくばバドレックスVMAX	14

アル_そらをとぶピカチュウ	258

キュレムVMAX	145

クロススイッチャー	257

カラマネロVMAX	6

オリジンパルキアVSTAR	246

ガラル マタドガス	301

LTB_空の封印石_other	6

LTB_ヤミラミ_リザードン	141

プテラVSTAR	37

アローラロコンVSTAR	52

ヒスイ ゾロアークVSTAR	45

アルセウス裏工作	58

ロトムVSTAR	21

レジドラゴVSTAR	8

ハピナスV	91

ミュウツーV-UNION	20

LOST_ギラティナVSTAR	187

オリジンディアルガVSTAR	25

LTB_リザードン	3

LTB_ウッウ	2

LTB_カイオーガ	163

LTB_カイリュー	47

LTB_空の封印石_水超鋼雷	82

LTB_空の封印石_草超鋼雷	5

LTB_空の封印石_水超闘雷	55

LTB_空の封印石_水超雷	14

LTB_空の封印石_水草超雷	2

LTB_空の封印石_水草超闘雷	2

LTB_空の封印石_水超鋼闘雷	30

LTB_空の封印石_水超鋼	3

LTB_空の封印石_超鋼雷	8

LTB_空の封印石_水草超	1

LTB_空の封印石_草超闘雷	1

LTB_空の封印石_草超鋼闘雷	4

LTB_空の封印石_水草超鋼雷	2

LTB_空の封印石_水草超鋼闘雷	1

LTB_空の封印石_超鋼闘雷	2

LTB_空の封印石_超闘雷	1

ダークライVSTAR	13

ミュウVMAX	599

ゾロア	65

ムゲンダイナVMAX	209

ヒスイ ダイケンキVSTAR	9

レジ	251

others	134

ゾロア_ウインディ	26



In [11]:
# number of decks for the target category
print(number_decks)
if number_decks <= 0:
    raise Exception(f"No decks for {TARGET_CATEGORY} during {START_DATE} to {END_DATE}")

33


In [12]:
# Analysis
df_list = []
common_cols = ["date", "prefecture", "num_players", "rank"]
int_cols = ["num_players", "rank"]

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[TARGET_CATEGORY]):
        deck = translate_deck(deck)

        deck_code = deck["deck_code"]  # row id
        if deck["date"] < START_DATE or deck["date"] > END_DATE:
            continue
        
        pokecard = OrderedDict()
        for col in common_cols:
            pokecard[col] = deck.get(col, "")
        pokecard.update(deck[card_type])

        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.fillna(0)

    # sort rows by date
    df = df.sort_values(by=["date"], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis="rows", numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis="rows")
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks

    # insert rows in df
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    target_rows = [num_rows - 1, num_rows - 2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]

    # sort cols by pick rate
    df = df.sort_values("pick_rate", axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)
    df = df[common_cols + col_list]

    # format data type
    for col in int_cols:
        df[col] = df[col].astype("Int64")

    # store
    df_list.append(df)

In [13]:
# excel writer
writer = pd.ExcelWriter(
    f"{EXCEL_FOLDER}/{TARGET_CATEGORY}/{TARGET_CATEGORY}-{START_DATE}-{END_DATE}.xlsx",
    engine="xlsxwriter",
    mode="w",
)
workbook = writer.book

for sheet_id, sheet_name in enumerate(
    ["pokemons", "tools", "supporters", "stadiums", "energies"]
):
    df_list[sheet_id].to_excel(writer, sheet_name=sheet_name, float_format="%.2f")

    # Make deck_link become a hyperlink
    # Get the xlsxwriter workbook and worksheet objects
    worksheet = writer.sheets[sheet_name]
    for i, deck_id in enumerate(df_list[sheet_id].index):
        if deck_id in ["avg_num_used", "pick_rate"]:
            continue

        # Calculate the row number
        row = i + 2

        # Write the hyperlink to the cell
        worksheet.write_url(
            f"A{row}",
            f"https://www.pokemon-card.com/deck/confirm.html/deckID/{deck_id}",
            string=f"{deck_id}",
        )

    # formatting
    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "top"}
    )
    for col_num, value in enumerate(df_list[sheet_id].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
    worksheet.autofit()

workbook.close()
writer.save()

In [14]:
df_list[0]

,date,prefecture,num_players,rank,瑪納霏\nS12a 029/172,光輝甲賀忍蛙\nS12a 033/172,ソルロック\nS10b 039/071,ルナトーン\nS10b 034/071,ナマコブシ\nS8 027/100,龍王蠍V\nS12a 085/172,ルナトーン\nS12a 056/172,ソルロック\nS12a 074/172,ディアンシー\nS9a 035/067,クレセリア\nSPD 008/020,ディアンシー\nS12a 063/172
pick_rate,NaN,NaN,<NA>,<NA>,1.0,1.0,0.757576,0.727273,0.545455,0.30303,0.272727,0.242424,0.151515,0.030303,0.030303
avg_num_used,NaN,NaN,<NA>,<NA>,1.0,1.0,4.000000,4.000000,1.000000,1.00000,4.000000,4.000000,1.000000,2.000000,1.000000
UXyypy-lUbxhT-MpXMyX,2023年01月14日(土),福島県,64,5,1.0,1.0,0.000000,0.000000,0.000000,1.00000,4.000000,4.000000,0.000000,0.000000,0.000000
NNLLHN-dGovHN-nLgngg,2023年01月14日(土),長崎県,64,11,1.0,1.0,4.000000,4.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000
9LgL6L-Sgx9N9-HnLnN9,2023年01月12日(木),東京都,70,1,1.0,1.0,4.000000,4.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
dk1VFv-R7MevZ-VkfdbF,2023年01月08日(日),愛知県,32,5,1.0,1.0,0.000000,0.000000,1.000000,1.00000,4.000000,4.000000,0.000000,0.000000,0.000000
FkkfkF-k81NKH-1dVfFV,2023年01月08日(日),愛知県,100,14,1.0,1.0,4.000000,4.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
XMp2XE-BwtHWx-UypMSy,2023年01月08日(日),静岡県,48,8,1.0,1.0,4.000000,4.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000
8Yx8Gx-gTAhY7-aY8xGD,2023年01月07日(土),埼玉県,100,5,1.0,1.0,4.000000,4.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
8G8cJJ-vVQaT0-DYJcxx,2023年01月07日(土),神奈川県,72,4,1.0,1.0,0.000000,0.000000,1.000000,0.00000,4.000000,4.000000,0.000000,0.000000,0.000000


In [15]:
df_list[1]

,date,prefecture,num_players,rank,洗翠的沉重球,回收網,レスキューキャリー,霧之水晶,先機球,レベルボール,講究腰帶,健行鞋,ともだちてちょう,タフネスマント,離洞繩,山谷回音喇叭,ポケギア3.0,げんきのハチマキ,普通釣竿,對戰VIP參加證,氣球,エネルギー回収,学習装置,ポケモンキャッチャー,放逐吸塵器
pick_rate,NaN,NaN,<NA>,<NA>,1.0,1.0,1.000000,1.000000,1.000000,0.878788,0.787879,0.727273,0.727273,0.727273,0.666667,0.424242,0.393939,0.333333,0.242424,0.121212,0.060606,0.060606,0.030303,0.030303,0.030303
avg_num_used,NaN,NaN,<NA>,<NA>,1.0,4.0,2.727273,2.727273,3.909091,2.965517,1.653846,2.291667,1.000000,1.500000,1.500000,1.000000,1.615385,1.545455,1.000000,3.750000,1.000000,1.000000,1.000000,1.000000,1.000000
UXyypy-lUbxhT-MpXMyX,2023年01月14日(土),福島県,64,5,1.0,4.0,3.000000,2.000000,4.000000,3.000000,0.000000,2.000000,0.000000,2.000000,2.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NNLLHN-dGovHN-nLgngg,2023年01月14日(土),長崎県,64,11,1.0,4.0,3.000000,3.000000,4.000000,3.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9LgL6L-Sgx9N9-HnLnN9,2023年01月12日(木),東京都,70,1,1.0,4.0,3.000000,3.000000,4.000000,2.000000,2.000000,1.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dk1VFv-R7MevZ-VkfdbF,2023年01月08日(日),愛知県,32,5,1.0,4.0,2.000000,4.000000,4.000000,4.000000,2.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkkfkF-k81NKH-1dVfFV,2023年01月08日(日),愛知県,100,14,1.0,4.0,3.000000,3.000000,4.000000,3.000000,2.000000,4.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
XMp2XE-BwtHWx-UypMSy,2023年01月08日(日),静岡県,48,8,1.0,4.0,3.000000,2.000000,4.000000,3.000000,2.000000,2.000000,0.000000,0.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8Yx8Gx-gTAhY7-aY8xGD,2023年01月07日(土),埼玉県,100,5,1.0,4.0,3.000000,2.000000,4.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8G8cJJ-vVQaT0-DYJcxx,2023年01月07日(土),神奈川県,72,4,1.0,4.0,3.000000,3.000000,4.000000,3.000000,1.000000,1.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000


In [16]:
df_list[2]

,date,prefecture,num_players,rank,莎莉娜,博士の研究,老大的指令,マリィ,キバナ,シロナの覇気,ツツジ
pick_rate,NaN,NaN,<NA>,<NA>,0.969697,0.939394,0.848485,0.727273,0.454545,0.121212,0.030303
avg_num_used,NaN,NaN,<NA>,<NA>,2.625000,3.322581,1.428571,1.666667,1.266667,1.250000,1.000000
UXyypy-lUbxhT-MpXMyX,2023年01月14日(土),福島県,64,5,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000
NNLLHN-dGovHN-nLgngg,2023年01月14日(土),長崎県,64,11,3.000000,4.000000,1.000000,0.000000,1.000000,0.000000,0.000000
9LgL6L-Sgx9N9-HnLnN9,2023年01月12日(木),東京都,70,1,4.000000,4.000000,0.000000,3.000000,0.000000,0.000000,0.000000
dk1VFv-R7MevZ-VkfdbF,2023年01月08日(日),愛知県,32,5,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,0.000000
FkkfkF-k81NKH-1dVfFV,2023年01月08日(日),愛知県,100,14,2.000000,4.000000,2.000000,0.000000,1.000000,0.000000,0.000000
XMp2XE-BwtHWx-UypMSy,2023年01月08日(日),静岡県,48,8,1.000000,3.000000,1.000000,2.000000,0.000000,1.000000,0.000000
8Yx8Gx-gTAhY7-aY8xGD,2023年01月07日(土),埼玉県,100,5,3.000000,4.000000,1.000000,1.000000,0.000000,0.000000,0.000000
8G8cJJ-vVQaT0-DYJcxx,2023年01月07日(土),神奈川県,72,4,2.000000,0.000000,1.000000,4.000000,2.000000,0.000000,0.000000


In [17]:
df_list[3]

,date,prefecture,num_players,rank,寶可補給站,シンオウ神殿,頂への雪道,いにしえの墓地,トレーニングコート
pick_rate,NaN,NaN,<NA>,<NA>,0.969697,0.121212,0.121212,0.030303,0.030303
avg_num_used,NaN,NaN,<NA>,<NA>,2.625000,1.500000,1.250000,1.000000,1.000000
UXyypy-lUbxhT-MpXMyX,2023年01月14日(土),福島県,64,5,3.000000,0.000000,0.000000,0.000000,0.000000
NNLLHN-dGovHN-nLgngg,2023年01月14日(土),長崎県,64,11,2.000000,0.000000,0.000000,0.000000,0.000000
9LgL6L-Sgx9N9-HnLnN9,2023年01月12日(木),東京都,70,1,2.000000,0.000000,0.000000,0.000000,0.000000
dk1VFv-R7MevZ-VkfdbF,2023年01月08日(日),愛知県,32,5,2.000000,1.000000,0.000000,0.000000,0.000000
FkkfkF-k81NKH-1dVfFV,2023年01月08日(日),愛知県,100,14,2.000000,0.000000,0.000000,0.000000,0.000000
XMp2XE-BwtHWx-UypMSy,2023年01月08日(日),静岡県,48,8,0.000000,2.000000,2.000000,0.000000,0.000000
8Yx8Gx-gTAhY7-aY8xGD,2023年01月07日(土),埼玉県,100,5,3.000000,0.000000,0.000000,0.000000,0.000000
8G8cJJ-vVQaT0-DYJcxx,2023年01月07日(土),神奈川県,72,4,2.000000,0.000000,0.000000,0.000000,1.000000


In [18]:
df_list[4]

,date,prefecture,num_players,rank,基本超能量,基本水能量
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,0.030303
avg_num_used,NaN,NaN,<NA>,<NA>,11.818182,2.000000
UXyypy-lUbxhT-MpXMyX,2023年01月14日(土),福島県,64,5,12.000000,0.000000
NNLLHN-dGovHN-nLgngg,2023年01月14日(土),長崎県,64,11,12.000000,0.000000
9LgL6L-Sgx9N9-HnLnN9,2023年01月12日(木),東京都,70,1,12.000000,0.000000
dk1VFv-R7MevZ-VkfdbF,2023年01月08日(日),愛知県,32,5,11.000000,0.000000
FkkfkF-k81NKH-1dVfFV,2023年01月08日(日),愛知県,100,14,12.000000,0.000000
XMp2XE-BwtHWx-UypMSy,2023年01月08日(日),静岡県,48,8,12.000000,0.000000
8Yx8Gx-gTAhY7-aY8xGD,2023年01月07日(土),埼玉県,100,5,11.000000,0.000000
8G8cJJ-vVQaT0-DYJcxx,2023年01月07日(土),神奈川県,72,4,10.000000,2.000000


In [19]:
len(decks["others"])

134